In [4]:
import os
import pydicom
import scipy
from scipy import ndimage

def get_data(dicom_dir):
    #resize the image to desired resolution
    xsize = 256; ysize = 256
    
    data = np.zeros((xsize, ysize, 100))
    for i, s in enumerate(os.listdir(dicom_dir)):
    
        img = np.array(pydicom.read_file(dicom_dir+ s).pixel_array)
        xscale = xsize/img.shape[0]
        yscale = ysize/img.shape[1]
        data[:,:,i] = scipy.ndimage.interpolation.zoom(img, [xscale, yscale])
    #returning a numpy array of shape 100,256,256
    return data


In [2]:
import pandas as pd
import numpy as np
df=pd.read_csv( '../vmshare/CTscan/overview.csv',encoding='utf-8',sep=',')
del df['Unnamed: 0']
y=df.iloc[:,1].values
y.shape

(100,)

In [5]:

X=get_data('../vmshare/CTscan/dicom_dir/')
X=np.moveaxis(X, -1, 0)
X.shape

(100, 256, 256)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test= train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
X_train.shape , y_train.shape, X_test.shape, y_test.shape

((80, 256, 256), (80,), (20, 256, 256), (20,))

In [11]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 25
num_classes = 2
epochs = 12


In [13]:
from autokeras import ImageClassifier

X_train = X_train.reshape(80,256,256,1)
X_test = X_test.reshape(20,256,256,1)

clf = ImageClassifier(verbose=True, augment=False)
clf.fit(X_train, y_train, time_limit= 60 * 60)
clf.final_fit(X_train, y_train, X_test, y_test, retrain=True)
y = clf.evaluate(X_test, y_test)
print(y * 100)


Initializing search.
Initialization finished.


╒==============================================╕
|               Training model 0               |
╘==============================================╛

Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   0.555080509185791    |          0.8           |
+--------------------------------------------------------------------------+


╒==============================================╕
|               Training model 1               |
╘==============================================╛

+--------------------------------------------------------------------------+
|    Father Model ID     |                 Added Operation                 |
+--------------------------------------------------------------------------+
|                      

MemoryError: 

In [15]:
clf.load_searcher().load_best_model().produce_keras_model().save('best_kerasautoML.h5')

In [19]:
test_predict=clf.predict(X_test)
test_predict=[1 if x else 0 for x in test_predict]
test_predict

[0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]

In [20]:
acc=0
for t, p in zip(y_test, test_predict):
    if t==p:
        acc+=1
    
print(float(acc/X_test.shape[0]))

0.65


In [17]:
from keras.models import load_model
model = load_model('best_kerasautoML.h5')


C:\Users\zecharpy\AppData\Local\Continuum\anaconda3\envs\opencv\lib\site-packages\keras\engine\saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
